# GANS are used to generate Periodic images 


# Imports for the program

In [ ]:
%matplotlib inline
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.utils.data as data
from torch import Tensor
from os import listdir
from os.path import join
import numpy as np
import h5py


# Defining the hyperparameters for the neural Network

In [ ]:
# Root directory for dataset
#dataroot = "data/celeba"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128
# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 1

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs =100

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1


# Defining  and loading the custom Dataset

In [ ]:
import torch.utils.data as data
from torch import Tensor
from os import listdir
from os.path import join
import numpy as np
#import h5py



def is_image_file(filename):
    return any(filename.endswith(extension) for extension in [".hdf5", ".h5"])


def load_img(filepath):
    img = None
    with h5py.File(filepath, "r") as f:
        img = f['data'][()]
    img = np.expand_dims(img, axis=0)
    torch_img = Tensor(img)
    torch_img = torch_img.div(255).sub(0.5).div(0.5)
    return torch_img

class HDF5Dataset(data.Dataset):
    def __init__(self, image_dir, input_transform=None, target_transform=None):
        super(HDF5Dataset, self).__init__()
        self.image_filenames = [join(image_dir, x) for x in listdir(image_dir) if is_image_file(x)]
        self.input_transform = input_transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        input = load_img(self.image_filenames[index])
        target = None

        return input

    def __len__(self):
        return len(self.image_filenames)

In [ ]:
## normalize the data to have mean of zero and standard deviation 1
transforms=transforms.Compose([
                               transforms.Resize(64),
                               transforms.CenterCrop(64),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5), (0.5)),
                           ])
dataset = HDF5Dataset(image_dir="training images",input_transform=transforms)
# dataset = HDF5Dataset(opt.dataroot,
                          
# comment mnist above and uncomment below if train on CelebA
#dataset = datasets.ImageFolder(root="celeb_dataset", transform=transforms)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [ ]:
# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
lab = get_truncated_normal(mean=0, sd=1, low=.7, upp=1.2)
labf=lab.rvs()
labs=get_truncated_normal(mean=0, sd=1, low=0, upp=0.3)
labsf=labs.rvs()
real_label = labf
fake_label = labsf
real_label=0.9 ## adding noise to stabilize the gan training
fake_label=0.1

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
import torch.nn.functional as F
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.conv1= nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0,bias=False)
        self.b1=nn.BatchNorm2d(ngf * 8)
        self.a1=nn.ReLU(True)
            # state size. (ngf*8) x 4 x 4
        self.conv2=nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2,padding=1, bias=False)
        self.b2=nn.BatchNorm2d(ngf * 4)
        self.a2=nn.ReLU(True)
            # state size. (ngf*4) x 8 x 8
        self.conv3=nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False)
        self.b3=nn.BatchNorm2d(ngf * 2)
        self.a3=nn.ReLU(True)
            # state size. (ngf*2) x 16 x 16
        self.conv4=nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False)
        self.b4=nn.BatchNorm2d(ngf)
        self.a4=nn.ReLU(True)
            # state size. (ngf) x 32 x 32
        self.conv5=nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False)
        self.tanh=nn.Tanh()
            # state size. (nc) x 64 x 64
        

    def forward(self, input):
        print(input.size())
        x=self.a1(self.b1(self.conv1(input)))
        print(x.size())
        
        x=self.a2(self.b2(self.conv2(x)))
        print(x.size())
        x=F.pad(x,[1,1,1,1],mode='circular')
        print('after padding size is {}'.format(x.size()))
        x=self.a3(self.b3(self.conv3(x)))
        print(x.size())
        x=F.pad(x,[1,1,1,1],mode='circular')
        print('after padding size is {}'.format(x.size()))
        x=self.a4(self.b4(self.conv4(x)))
        print(x.size())
        
        x=self.tanh(self.conv5(x))
        print(x.size())
        return x

# Training Loop

In [ ]:
# Training Loop

# Lists to keep track of progress
lr = 0.00002 ## training again with less learning rate 
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):
        
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data.to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        #for p in list(filter(lambda p: p.grad is not None, netD.parameters())):
           # print(p.grad.data.norm(2).item())
        
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        fake = netG(noise)
        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        
        errD_fake.backward()
        #for p in list(filter(lambda p: p.grad is not None, netD.parameters())):
             #print(p.grad.data.norm(2).item())
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        #for p in list(filter(lambda p: p.grad is not None, netG.parameters())):
                # print(p.grad.data.norm(2).item())
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()
        
        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        
        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())
        
        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))
            
        iters += 1

## loading the required image processing Tools

In [ ]:

import torch
import numpy as np
import h5py
import tifffile
from scipy.ndimage.filters import median_filter
from skimage.filters import threshold_otsu
from collections import Counter


In [ ]:
modell = Generator(1)
modell.load_state_dict(torch.load('model_weights3.pth'))

In [ ]:
noise_test = torch.randn(128, nz, 1, 1)
test1=modell(noise_test)

In [ ]:
generated_images

In [ ]:

import torch
import numpy as np
import h5py
import tifffile
from scipy.ndimage.filters import median_filter
from skimage.filters import threshold_otsu
from collections import Counter
b_size=128
generated_images=[]
for index in range(5):
    noise_test = torch.randn(b_size, nz, 1, 1)
    test1=modell(noise_test)
    test1_final=test1[0].reshape(88,88).cpu().detach().numpy()
    generated_images.append(test1_final)

figure=plt.figure(figsize=(15,7))
for index,image in enumerate(generated_images):
    plt.subplot(1,5,index+1)
    plt.title('synthetic_image'+str(index))
    plt.axis('off')


    test1_final = median_filter(image,size=(3,3))
    #test1_final = median_filter(test1_final,size=(22))

    threshold_global_otsu = threshold_otsu(test1_final)

    segmented_image = (test1_final >= threshold_global_otsu).astype(np.int32)

    plt.imshow(segmented_image,cmap='Greys')
 
    
    
